In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time as T
import cPickle
%matplotlib inline

In [2]:
file_1=open('cifar_10_py/data_batch_1')
file_2=open('cifar_10_py/data_batch_2')
file_3=open('cifar_10_py/data_batch_3')
file_4=open('cifar_10_py/data_batch_4')
file_5=open('cifar_10_py/data_batch_5')

from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)
x_batch, y_true_batch = data.train.next_batch(1060000)
x_test,y_test_tru=data.test.next_batch(100000)

In [3]:
train_1=cPickle.load(file_1)
train_2=cPickle.load(file_2)
train_3=cPickle.load(file_3)
train_4=cPickle.load(file_4)
train_5=cPickle.load(file_5)

In [4]:
train=np.array(train_1['data'])
train=np.concatenate((train,train_2['data']),0)
train=np.concatenate((train,train_3['data']),0)
train=np.concatenate((train,train_4['data']),0)
train=np.concatenate((train,train_5['data']),0)

train=np.transpose(train.reshape([-1,3,32,32]), (0,2, 3,1))
train=train.reshape([-1,3*32*32])

train_lab=np.array(train_1['labels'])
train_lab=np.concatenate((train_lab,train_2['labels']),0)
train_lab=np.concatenate((train_lab,train_3['labels']),0)
train_lab=np.concatenate((train_lab,train_4['labels']),0)
train_lab=np.concatenate((train_lab,train_5['labels']),0)

In [5]:
file_=open('cifar_10_py/test_batch')
test_x=cPickle.load(file_)

In [6]:
train_1.keys()

['data', 'labels', 'batch_label', 'filenames']

In [7]:
y_lab = {0 : 'airplane',1 : 'automobile' ,2 : 'bird',3 : 'cat',4 : 'deer',5 : 'dog',
         6 : 'frog', 7 : 'horse' ,8 : 'ship' ,9 : 'truck'}

In [39]:
confil1 = 5
nofil1 = 16

confil2 = 6
nofil2 = 36

confil3 = 9
nofil3 = 72

confil4 = 9
nofil4 = 72

fc1 = 300
fc2 = 300
fc3 = 10


In [9]:
input_no= 32*32*3
output_no = 10

In [10]:
img_size = 32
img_shape = (img_size,img_size)
no_channels = 3
seed =234

In [11]:
x = tf.placeholder(tf.float32,[None,input_no])
y = tf.placeholder(tf.float32,[None,output_no])

In [12]:
def new_weights(shape,nam):
    return tf.Variable(tf.random_normal(shape,seed=seed,name=nam))

In [13]:
def new_biases(length,nam):
    return tf.Variable(tf.random_normal([length],seed=seed,name=nam))

In [38]:
def new_conv(input , num_cha , size , no_fils , nam, use_maxpool=True):
    shape = [ size , size , num_cha , no_fils ]
    weights = new_weights( shape = shape,nam=nam )
    bias = new_biases( length = no_fils,nam=nam )
    layer = tf.nn.conv2d(input=input,filter =weights,strides=[1,1,1,1],padding='SAME')
    layer += bias
    if use_maxpool:
        layer = tf.nn.max_pool(value=layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    layer = tf.nn.relu(layer)
    return layer
    

In [15]:
def flatten(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

In [16]:
def new_fc(input, no_inpu,no_out,nam,isrelu=True,keep_pro=1.0):
    shape=[no_inpu,no_out]
    weights= new_weights(shape,nam)
    length=no_out
    bias= new_biases(length,nam)
    layer= tf.matmul(input,weights)+bias
    if isrelu:
        layer=tf.nn.relu(layer)
    layer=tf.nn.dropout(layer,keep_pro)
    return layer

In [17]:
x_image=tf.reshape(x,[-1,img_size,img_size,no_channels])

In [18]:
kee_prob= tf.placeholder(tf.float32)

In [40]:
lay_conv1= new_conv(input= x_image,num_cha= no_channels,size=confil1,no_fils=nofil1,nam="conv_lay1",use_maxpool=False)

In [41]:
lay_conv2=new_conv(input=lay_conv1,num_cha= nofil1,size=confil2,no_fils=nofil2,nam="conv_lay2")

In [42]:
lay_conv3=new_conv(input=lay_conv2,num_cha=nofil2,size=confil3,no_fils=nofil3,nam="conv_lay3",use_maxpool=False)

In [43]:
lay_conv4=new_conv(input=lay_conv3,num_cha=nofil3,size=confil4,no_fils=nofil4,nam="conv_lay4")

In [44]:
flat,num_fe=flatten(lay_conv4)

In [45]:
fc_lay1=new_fc(input=flat,no_inpu=num_fe,no_out=fc1,keep_pro=kee_prob,nam="fc1")

In [46]:
fc_lay2=new_fc(input=fc_lay1,no_inpu=fc1,no_out=fc2,keep_pro=kee_prob,nam="fc2")

In [47]:
fc_lay3=new_fc(input=fc_lay2,no_inpu=fc2,no_out=output_no,keep_pro=kee_prob,isrelu=False,nam="final")

In [48]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc_lay3,labels=y))

In [49]:
pred_i=tf.nn.softmax(fc_lay3)
pred_cls=tf.argmax(pred_i,axis=1)

In [50]:
learning_rate= 0.001
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [51]:
init=tf.global_variables_initializer()

In [52]:
sess=tf.Session()

In [53]:
sess.run(init)

In [54]:
c_su=0
ju=10
lk=5
le=50000/ju
t=T.clock()
for jf in xrange(lk):
    for i in xrange(le):
        x_in=train[i*ju:(i+1)*ju]
        y_sese=np.zeros([ju,10])
        ke=train_lab[i*ju:(i+1)*ju]
        for j in range(ju):
            y_sese[j,ke[j]]=1

        _,c=sess.run([optimizer,cost],feed_dict={x:x_in, y:y_sese, kee_prob:1.0})
        c_su+=c
        if (i+1) % 500==0:
            print jf+1," ",i+1," ",c_su/500
            c_su=0
print T.clock()-t

1   500   33652086544.4
1   1000   8483010592.26
1   1500   4541077182.72
1   2000   2886135406.98
1   2500   1840112261.06
1   3000   1273100991.55
1   3500   900748613.856
1   4000   614033747.264
1   4500   467520848.96
1   5000   335055562.832
2   500   266404110.88
2   1000   195145765.128
2   1500   148414968.412
2   2000   112195563.456
2   2500   85304329.112
2   3000   60934950.22
2   3500   46601338.266
2   4000   34950566.727
2   4500   26673765.874
2   5000   20842908.4205
3   500   17508833.4765
3   1000   13886015.4315
3   1500   11877491.623
3   2000   9902212.16325
3   2500   7607983.66675
3   3000   7328549.64925
3   3500   6101190.8005
3   4000   5125017.03463
3   4500   4514069.13825
3   5000   3962923.97775
4   500   3237947.1775
4   1000   2646764.35106
4   1500   2309307.1045
4   2000   1861459.82756
4   2500   1446601.09519
4   3000   1560275.12181
4   3500   1070068.99622
4   4000   980834.4455
4   4500   932721.778219
4   5000   817064.902437
5   500   670788.9

In [ ]:
y_sese

In [33]:
testx=np.array(test_x['data'])
lab_y=np.array(test_x['labels'])

In [34]:
testx=np.transpose(testx.reshape([-1,3,32,32]), (0,2, 3,1))

testx=testx.reshape([-1,3*32*32])

In [55]:
t=T.clock()
ac_su=0
print "testing the model"
for ki in xrange(10000):
    pred=sess.run(pred_cls,{x:testx[ki:ki+1],kee_prob:1.0})
    if pred==np.argmax(lab_y[ki]):
        ac_su+=1
    #if (ki+1)%100==0:
        #print "acc:",(ac_su*100.0)/ki,
print ac_su
print T.clock()-t

testing the model
6
863.182481


In [56]:
print "acuracy: "
ac_su/10000.0

0.0006

In [ ]:
i=253
t=T.clock()
pred=sess.run(fc_lay3,{x:train[i:i+1],kee_prob:1.0})
print y_lab[np.argmax(pred)],y_lab[train_lab[i]]
print T.clock()-t
plt.imshow(train[i].reshape(32,32,3))
#plt.imshow(testx[1].reshape(32,32,3))

In [ ]:
pred

In [ ]:
y_sese

In [ ]:
plt.imshow(train[i].reshape(32,32,3))

In [ ]:
y_lab[8]

In [ ]:
ac_su

In [ ]:
#hi=np.transpose(train.reshape([-1,3,32,32]), (0,2, 3,1))

In [ ]:
#train=hi.reshape([-1,32*32*3])

In [ ]:
io=194
tra = testx[io].reshape([32,32,3])
print y_lab[lab_y[io]]
plt.imshow(tra)

In [ ]:
hi.shape

In [ ]:
3638180026.5/500

In [ ]:
writer = tf.summary.FileWriter('cnn_cifar')  
writer.add_graph(tf.get_default_graph())


In [ ]:
(464-131)/6